# Imports - DO NOT TOUCH, just hit run all (play button)

In [ ]:
!pip install orjson
!pip install pytrends

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import orjson
import datetime
from datetime import timedelta
from pytrends.request import TrendReq
from dateutil import parser
from tqdm import tqdm
import warnings
from functools import reduce
from time import sleep
import ast
import statistics as s
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore
from scipy import stats
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
import joblib
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 2.4 MB/s eta 0:00:00


# Data Collection - DO NOT TOUCH, just hit run all (play button)

In [ ]:
# Box Office

# Box Office: get_box_office(df, session) -> df, join on 'url'

def scrape_box(url, session):
    r = session.get(url, headers={'User-Agent':'Mozilla/5.0'})
    soup = BeautifulSoup(r.text, 'lxml')
    a = soup.find('table', id='movie_finances')
    pop = a.find_all('tr', class_='heading')
    if pop != []:
      for x in pop:
        x.extract()

    fin = a.find_all('td')

    b = soup.find('div', class_='card-body')
    c = b.find_all('table')
    d = c[1]
    rows = d.find_all('td')
    all = fin + rows

    p = {'Opening\xa0Weekend:' : 'opening',
         'Legs:' : 'legs',
         'Domestic Share:' : 'share',
         'Production\xa0Budget:' : 'prod',
         'Theater counts:' : 'theater',
         'Infl. Adj. Dom. BO' : 'adjusted',
         'Domestic Box Office': 'domestic_total',
         'International Box Office' : 'international_total',
         'Worldwide Box Office' : 'all_total',
         'Est. Domestic DVD Sales' : 'domestic_dvd',
         'Est. Domestic Blu-ray Sales' : 'domestic_bluray',
         'Total Est. Domestic Video Sales': 'domestic_home'
         }

    dicto = {
      'url': url,
      'official_name': soup.find('div', id='main').find('h1').text,
      'opening': None,
      'share': None,
      'legs': None,
      'prod':  None,
      'theater': None,
      'adjusted': None,
      'domestic_total': None,
      'international_total': None,
      'all_total': None,
      'domestic_dvd': None,
      'domestic_bluray': None,
      'domestic_home': None
    }

    for x in range(len(all)):
      iter = all[x].text
      if iter in p.keys():
        dicto[p[iter]] = all[x+1].text

    return dicto

def cut(text):
  if text is not None:
    text = str(text)
    pattern = r'\([^)]*\)'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text
  else:
    return None

def extract(text):
  text = str(text)
  pattern = r'\((.*?)\)'
  match = re.search(pattern, text)
  if match:
      return match.group(1)
  return None

def number(text):
  pattern = r'\d+(\.\d+)?'
  text = str(text)
  match = re.search(pattern, text)
  if match:
      number = match.group()
      return float(number) if '.' in number else int(number)
  return None

def theatre(text):
  text = str(text)
  pattern = r'\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b'
  numbers_with_commas = re.findall(pattern, text)
  numbers = [float(num.replace(',', '')) if '.' in num else int(num.replace(',', '')) for num in numbers_with_commas]
  return numbers

def format(text):
  if text is None:
    return None
  elif '$' in text:
    text = text.replace('$', '')
    if ',' in text:
      text = text.replace(',', '')
    return float(text)
  elif 'n/a' in text:
    return None

def alter_df(df1):
  for x in ['share', 'legs', 'opening', 'prod']:
    df1[x] = df1[x].apply(cut)

  df1['opening_percentage'] = df1['opening'].apply(extract)
  df1['prod_multiple'] = df1['prod'].apply(extract)

  for x in ['prod_multiple', 'opening_percentage', 'share']:
    df1[x] = df1.apply(number)

  df1['theater'] = df1['theater'].apply(theatre)
  df1['opening_theater'] = df1['theater'].apply(lambda x: None if len(x) < 3 else x[0])
  df1['max_theater'] = df1['theater'].apply(lambda x: None if len(x) < 3 else x[1])
  df1['average_theater_run'] = df1['theater'].apply(lambda x: None if len(x) < 3 else x[2])
  df1 = df1.drop('theater', axis=1)

  for x in ['opening', 'prod', 'domestic_total', 'international_total', 'all_total', 'domestic_dvd', 'domestic_bluray', 'domestic_home']:
    df1[x] = df1[x].apply(format)

  return df1

def get_box_office(df, session):
  tqdm.pandas(desc='{Box Office Pull}')
  df['data'] = df['the_numbers_url'].progress_apply(lambda x: scrape_box(x, session))
  un_df = pd.json_normalize(df['data'])
  formatted = alter_df(un_df)
  formatted = formatted['title', 'opening', 'domestic_adjusted', 'domestic_total', 'international_total', 'international_adjusted', 'all_total', 'all_adjusted', 'domestic_with_home']
  return formatted

In [ ]:
# Wikipedia

def filter_it(subset):
  if len(subset) > 90:
    a = {}
    a['wiki_earliest_set_pure'] = subset[-90:-60]
    a['wiki_earliest_set_com'] = subset[-90:]
    a['wiki_mid_set_pure'] = subset[-60:-30]
    a['wiki_mid_set_com'] = subset[-60:]
    a['wiki_latest_set'] = subset[-30:]
    a['wiki_all_data'] = subset
    return a
  elif len(subset) > 60:
    a = {}
    a['wiki_earliest_set_pure'] = None
    a['wiki_earliest_set_com'] = subset[-90:]
    a['wiki_mid_set_pure'] = subset[-60:-30]
    a['wiki_mid_set_com'] = subset[-60:]
    a['wiki_latest_set'] = subset[-30:]
    a['wiki_all_data'] = subset
  elif len(subset) > 30:
    a = {}
    a['wiki_earliest_set_pure'] = None
    a['wiki_earliest_set_com'] = None
    a['wiki_mid_set_pure'] = None
    a['wiki_mid_set_com'] = subset[-60:]
    a['wiki_latest_set'] = subset[-30:]
    a['wiki_all_data'] = subset
  else:
    a = {}
    a['wiki_earliest_set_pure'] = None
    a['wiki_earliest_set_com'] = None
    a['wiki_mid_set_pure'] = None
    a['wiki_mid_set_com'] = None
    a['wiki_latest_set'] = None
    a['wiki_all_data'] = subset
  return a

def get_creation(wikipedia_link, session):
  title = wikipedia_link.split('/')[-1]
  revisions_url = f'https://en.wikipedia.org/w/api.php?action=query&format=json&titles={title}&prop=revisions&rvprop=timestamp&rvlimit=1&rvdir=newer'
  response = session.get(revisions_url, headers={'User-Agent':'Mozilla/5.0'})
  data = response.json()
  page_id = list(data['query']['pages'].keys())[0]
  revision_date_str = data['query']['pages'][page_id]['revisions'][0]['timestamp']
  revision_date_obj = datetime.datetime.strptime(revision_date_str, '%Y-%m-%dT%H:%M:%SZ')
  return revision_date_obj.strftime('%Y%m%d')

def get_release(wikipedia_link, session):
  try:
    response = session.get(wikipedia_link, headers={'User-Agent': 'Mozilla/5.0'})
    page_content = response.text
    soup = BeautifulSoup(page_content, 'html.parser')
    table = soup.find('table', class_='infobox vevent')
    rows = soup.find_all('tr')

    for row in rows:
      if "Release dates" in row.text:
        listy = row.text.strip().split('\n')
        for item in listy:
          if 'United States' in item or 'US' in item:
            result = re.sub(r'\([^)]*\)', '', item)
            result = result.replace('\xa0', ' ').strip()
            parsed_date = parser.parse(result)
            return parsed_date.strftime('%Y%m%d')
        for item in listy:
          if '(' in item:
            result = re.sub(r'\([^)]*\)', '', item)
            result = result.replace('\xa0', ' ').strip()
            parsed_date = parser.parse(result)
            return parsed_date.strftime('%Y%m%d')

      elif 'Release date' in row.text:
        listy = row.text.strip().split('\n')
        for item in listy:
          if '(' in item:
            result = re.sub(r'\([^)]*\)', '', item)
            result = result.replace('\xa0', ' ').strip()
            parsed_date = parser.parse(result)
            return parsed_date.strftime('%Y%m%d')
  except:
    return None

def api_call(url, first_date, end_date, session):
    end_part = url.split('/')[-1]
    api = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{end_part}/daily/{first_date}/{end_date}'
    resp = session.get(api, headers={'User-Agent':'Mozilla/5.0'})
    json_data = orjson.loads(resp.content)

    start = datetime.datetime.strptime(first_date, '%Y%m%d')
    stop = datetime.datetime.strptime(end_date, '%Y%m%d')

    views_dict = {item['timestamp'][:8]: item['views'] for item in json_data['items']}

    current_date = start
    j_data = []
    while current_date <= stop:
        j_data.append(views_dict.get(current_date.strftime('%Y%m%d'), 0))
        current_date += datetime.timedelta(days=1)

    return j_data

def get_wiki(link, session, timestamp):
  release = get_release(link, session)
  creation = get_creation(link, session)
  data = api_call(link, creation, release, session)
  if timestamp == 90:
    x = filter_it(data[:-90])
  elif timestamp == 60:
    x = filter_it(data[:-60])
  elif timestamp == 30:
    x = filter_it(data[:-30])
  else:
    raise ValueError('Not valid timestamp entry.')

  x['wikipedia_url'] = link
  x['release'] = release
  return x


In [ ]:
def ready_title(text):
  title = re.sub(r'\([^)]*\)', '', text)
  title = title.strip()
  if len(title) >= 100:
    film_title = film_title[:99]
    a = film_title.rfind(':')
    if a == -1:
      return film_title[:a]
    else:
      return film_title[:a]
  else:
    return title

def get_trends_data(film_title, release, t, timestamp):
  a = {'title': film_title}
  film_title = ready_title(film_title)
  release = datetime.datetime.strptime(release, "%Y%m%d")
  end = release - pd.DateOffset(days = timestamp)

  times = {
      'gt_earliest_set_pure': (90, 0),
      'gt_earliest_set_raw': (90, 60),
      'gt_mid_set_pure': (60, 30),
      'gt_mid_set_com': (60, 0),
      'gt_latest_set': (30,0)
    }

  for key, item in tqdm(times.items(), desc=f'Google Trend Pulls for {film_title}'):
    start_date = (end - datetime.timedelta(item[0])).strftime('%Y-%m-%d')
    end_date = (end - datetime.timedelta(item[1])).strftime('%Y-%m-%d')
    retry = True
    while retry:
      try:
        t.build_payload(kw_list=[film_title], timeframe=f'{start_date} {end_date}')
        trends_data = t.interest_over_time()
        trends_data = trends_data.rename(columns={film_title: 'film'})

        try:
          a[key] = trends_data['film'].to_list()
          sleep(1)
          break

        except KeyError as e:
          print(e)
          index = film_title.rfind(':')
          if index == -1:
            film_title = film_title[: len(film_title) // 2]
          else:
            film_title = film_title[:index]

      except Exception as e:
        print(e,  ': rate limit hit - relaunching pull for', film_title)
        sleep(60)

  return a

In [ ]:
def master_data(df, timeframe):
  s = requests.Session()
  t = TrendReq()

  tqdm.pandas(desc='{Wikipedia Data Pull}')
  df['wiki_data'] = df['wikipedia_url'].progress_apply(lambda x: get_wiki(x, s, timeframe))
  wiki_df = pd.json_normalize(df['wiki_data'])
  df = pd.merge(df, wiki_df, how='inner', on='wikipedia_url').drop(columns=['wiki_data'])

  tqdm.pandas(desc='{Google Trends Pull - may take multiple iterations}')
  df['gt_data'] = df.progress_apply(lambda x: get_trends_data(x['title'], x['release'], t, timeframe), axis=1)
  gt_df = pd.json_normalize(df['gt_data'])
  df = pd.merge(df, gt_df, how='inner', on='title').drop(columns=['gt_data'])

  return df

# Data Analysis - DO NOT TOUCH, just hit run all (play button)

In [ ]:
# Statistical analysis

def normalize_list(input_list):
    scaler = MinMaxScaler(feature_range=(0, 100))
    normalized_list = scaler.fit_transform([[x] for x in input_list])
    return normalized_list.ravel().tolist()

def temp_solution(text):
  try:
    return ast.literal_eval(str(text))
  except ValueError:
    if str(text).lower() == "nan":
      return None  # or you can return "nan" or any other placeholder value
    else:
      raise

def try_std(x):
  try:
    return s.stdev(x) if x is not None else None
  except ValueError:
    return None

def analyze(df):
  df = df.copy()

  transformations = [('sum', lambda x: sum(x)  if x is not None else None),
                     ('avg', lambda x: s.mean(x) if x is not None else None),
                     ('std', try_std),
                      #lambda x: s.stdev(x) if x is not None else None),
                     ('median', lambda x: s.median(x)  if x is not None else None),
                     ('mode', lambda x: s.mode(x)  if x is not None else None),
                     ('range', lambda x: max(x) - min(x)  if x is not None else None),
                     ('iqr', lambda x:  s.quantiles(x, n=4)[-1] - (s.quantiles(x, n=4)[0])  if x is not None else None),
                    ]

  '''
  for column in df.drop(columns=['title', 'release']):

    if ('skew' not in column) and ('kurtosis' not in column):
      for x in transformations:
        df[column + '_' + x[0]] = df[column].apply(x[1])
    df = df.drop(column, axis=1)
  '''

  new_cols = []
  drop_col = []
  for column in df.drop(columns=['title', 'release', 'date']):
      if ('skew' not in column) and ('kurtosis' not in column):
          temp_df = pd.DataFrame()
          for x in transformations:
            temp_df[column + '_' + x[0]] = df[column].apply(x[1])
            drop_col.append(column)
          new_cols.append(temp_df)

  df = pd.concat([df, *new_cols], axis=1)
  df = df.drop(columns = drop_col)

  return df

def expand(df):
  df = df.copy()
  df = df.drop('wikipedia_url', axis=1)
  df['wiki_all_data'] = df['wiki_all_data'].apply(lambda x: None if x == [] else x)
  first = df.drop(columns=['title', 'release', 'date'])

  for column in first:
    #df[column + '_skew'] = df[column].apply(lambda x: stats.skew(x) if x is not None else None)
    #df[column + '_kurtosis'] = df[column].apply(lambda x: stats.kurtosis(x) if x is not None else None)
    if 'wiki' in column:
      df[column + '_norm'] = df[column].apply(lambda x: normalize_list(x) if x is not None else None)

  df = analyze(df)

  return df.copy()

# Machine Learning - DO NOT TOUCH, just hit run all (play button)

In [ ]:
ml_models = {
    "Decision Tree": DecisionTreeRegressor(),
    "Extra Trees": ExtraTreesRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Random Forest": RandomForestRegressor()
}

param_grids = {
    "Decision Tree": {
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    },

    "Extra Trees": {
        'n_estimators': [50, 100],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    },

    "Gradient Boosting": {
        'n_estimators': [50, 100],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'subsample': [0.8, 1.0]
    },

    "Random Forest": {
        'n_estimators': [50, 100],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
}

def best_predictive_models(input_df, output_df):
  """Given an input and output DataFrame, returns the best models by R^2 score."""

  def preprocess_data(input_df, output_df, column):
    """Prepare data for training."""
    merged_df = pd.merge(input_df, output_df, left_on='title', right_on='title', how='inner')
    merged_df.fillna(merged_df.median(numeric_only=True), inplace=True)
    X = merged_df.drop(columns=['title', 'release'] + list(output_df.columns))
    y = merged_df[column]
    return X, y

  def get_best_model_for_preprocessed_data(X, y, n_iter=10):
        """Find the best model for the preprocessed data."""
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
        best_r2 = float('-inf')
        best_model_info = None

        for model_name, model_instance in ml_models.items():
            grid_search = RandomizedSearchCV(model_instance, param_grids[model_name], n_iter=n_iter, cv=5, scoring='r2', n_jobs=-1, random_state=4)
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            y_pred = best_model.predict(X_test)
            r2 = r2_score(y_test, y_pred)
            if r2 > best_r2:
                best_r2 = r2
                best_model_info = {
                    'Model': model_name,
                    'Instance': best_model,
                    'Best Hyperparameters': grid_search.best_params_,
                    'R^2': r2
                }
        return best_model_info

  # Dictionary to store the results and trained models
  results = {}
  trained_models = {}

  for column in output_df.columns.drop("title"):
      X, y = preprocess_data(input_df, output_df, column)
      result = get_best_model_for_preprocessed_data(X, y)
      results[column] = f"{result['Model']} with R^2: {result['R^2']}"
      # Modified the following line to store a tuple as described
      trained_models[column] = (result['Instance'], result['R^2'], result['Best Hyperparameters'])
      print(f"Best Model for {column}: {results[column]}\n")

  return trained_models

Best Model for opening: Gradient Boosting with R^2: 0.7680471361032073

Best Model for domestic_adjusted: Gradient Boosting with R^2: 0.38928213220986074

Best Model for domestic_total: Gradient Boosting with R^2: 0.338594694140509

Best Model for international_total: Random Forest with R^2: 0.8150215420261888

Best Model for international_adjusted: Random Forest with R^2: 0.7389423616872677

Best Model for domestic_with_home: Gradient Boosting with R^2: 0.5061498291170246

Best Model for all_total: Random Forest with R^2: 0.8485818583945088

Best Model for all_adjusted: Gradient Boosting with R^2: 0.7069531582042514

Best Model for domestic_home: Random Forest with R^2: 0.5476382398811566

Best Model for opening: Random Forest with R^2: 0.7228369900343856

Best Model for domestic_adjusted: Extra Trees with R^2: 0.1360661754252257

Best Model for domestic_total: Gradient Boosting with R^2: 0.07154503546113611

Best Model for international_total: Extra Trees with R^2: 0.856727893244689


In [ ]:
def save(dictionary, metric, time):
  filename = f'{time}_{metric}.joblib'
  joblib.dump(dictionary[metric], '/content/' + filename)
  return filename

'\n!git clone https://github.com/nathanchong25/film_ml\n%cd film_ml\n!mv /content/myfile.txt /content/film_ml/\n!git config --global user.email "nathanchong25@gmail.com"\n!git config --global user.name "nathanchong25"\n\nfor file in files_to_push:\n    !git add {file}\n    !git commit -m "Added/Updated {file}"\n\n!git push origin master\n'

In [ ]:
def recalibrate(df):
  df30n = expand(master_data(df[['title', 'wikipedia_url']], 30))
  df60n = expand(master_data(df['title', 'wikipedia_url'], 60))
  df90n = expand(master_data(df['title', 'wikipedia_url'], 90))
  s = requests.Session()
  output = get_box_office(df[['title', 'the_numbers_url']])
  a = best_predictive_models(df30n, output)
  b = best_predictive_models(df60n, output)
  c = best_predictive_models(df90n, output)
  return (a, b, c)

def main():
  file_test = True
  while file_test:
    filepath = input('Enter filename (ensure file is uploaded in Files on the left, enter as file.csv or file.xlsx): ')

    if filepath[-4:] == '.csv':
      try:
        df = pd.read_csv('/content/' + filepath)
        if 'Unnamed: 0' in set(df.columns):
          df = df.drop('Unnamed: 0', axis = 1)
        elif '' in set(df.columns):
          df = df.drop('', axis = 1)
        if set(df.columns) == {'title', 'wikipedia_url', 'the_numbers_url'}:
          file_test = False
        else:
          print('Table columns are incorrect - check input file is in the correct format.')
      except FileNotFoundError:
        print('File not found - check filename and ensure file is uploaded in the right location (content folder).')
      except:
        print('Problem reading input file - check input file is in the correct format.')

    elif filepath[-5:] == '.xlsx':
      try:
        warnings.warn('Excel file must only be one sheet.')
        df = pd.read_excel('/content/' + filepath)
        if 'Unnamed: 0' in set(df.columns):
          df = df.drop('Unnamed: 0', axis = 1)
        elif '' in set(df.columns):
          df = df.drop('', axis = 1)
        if set(df.columns) == {'title', 'wikipedia_url', 'the_numbers_url'}:
          file_test = False
        else:
          print('Table columns are incorrect - check input file is in the correct format.')
      except FileNotFoundError:
        print('File not found - check filename and ensure file is uploaded in the right location (content folder).')
      except ValueError:
        print('Problem reading input file - ensure that the Excel file is only one sheet.')
      except:
        print('Problem reading input file - check input file is in the correct format.')
    else:
       print('Faulty input - program only accepts CSV (.csv) and Excel (.xlsx) files.')

    a = recalibrate(df)

    global files_to_push
    files_to_push = []
    for x in a:
      for y in x:
        for z in [30, 60, 90]:
          files_to_push.append(save(x, y, z))

    return a


# User Interface

In [ ]:
files_to_push = []
main()

KeyboardInterrupt: ignored

In [ ]:
# Do not run this unless you are sure you know what you are doing and the models are a sure improvement. Running this will replace everything.

!git clone https://github.com/nathanchong25/film_ml
%cd film_ml
!mv /content/myfile.txt /content/film_ml/
!git config --global user.email "nathanchong25@gmail.com"
!git config --global user.name "nathanchong25"

for file in files_to_push:
    !git add {file}
    !git commit -m "Added/Updated {file}"

!git push origin master